# Recommender Systems Intro

Below are the notes and implementation for some concepts used in builing recommendation algorithms.

### Popularity

For instance, movies in the  top 10 list of Netflix. 

Problem - highest popularity does not mean best choice - 0 exploration, full exploitation

For instance music platforms, news websites - e.g., Age, location and other factors play important role  

### Product associations

 e.g., Complement products - products that logically go together, context matters in recommendations. - People who buy phone usually also by phone cover, headphones etc.

 * `Lyft` -  prob that a and b divided by prod of pa and pb
 $$Lyft = \frac{Pr(a \text{ and } b)}{ Pr(a)Pr(b)} = \frac{Pr(a|b)}{Pr(a)} = \frac{Pr(b|a)}{Pr(b)}$$ 

 * symmetric metric
 * if a and b is independent, we get $Lyft = 1$
 * if buying b increases the likelihood of buying a then lyft>1.

### Hacker News

* Ratio of popularity to time (age) $$score = \frac{(ups-downs -1)^{0.8}}{(age+2)^{gravity}} * penalty $$

* gravity = 1.8 - how fast the score goes down with age (in hours)

* penalty - sets the business rule (for instance, self posts ranked lower than the post containing the link to original source)

* popularity follow power law (long tail) - few articles have high votes, and most have fewer votes - so popularity grows sublinearly

### Problems with average Ratings

Binary, range ratings 

Sorting by average rating - we need to be confident in the rating as well. Calculate confidence interval upper and lower bound ---> pessimistic approach use lower bound



$$\bar{X} \sim N(\mu, \frac{\sigma^2}{N})$$

Higher rating count  => narrower confidence interval 

Even though data does not follow normal distribution, by Central Limit theorem sum of random variables follow normal distribution

Binary ratings, we can use Bernoilly distribution. 
$$ \hat{p} = \frac{\# success}{N}$$
$$95CI = [\hat{p}\pm 1.96 * \sqrt{\frac{\hat{p}(1-\hat{p})}{N}}]$$

Better to use Wilson confidence interval (read the extra readings file)

* zero ratings problem - no average - use smoothing (dampening) by adding a small number to numerator and denominator, or use a medium rating as 3 stars.

$$ Score = \frac{\sum X + \mu*\lambda}{N+\lambda} $$

* The above formula, scales down the review if the number of reviews is smaller. For instance, for $\lambda = 1$, a product with zero reviews will have mean review initially. A product with very few bad reviews will have higher than the average rating, and a product with very few but good reviews will have a review smaller than average reviews, in other words initially the algorithm keeps the score closer to the average rating until it has enough confidence calculated with many reviews.

### Explore - Exploit dilemma

For instance, Good estimate requires more data, and on the other hand more data means more data to spend to do something suboptimal. Suppose that youtube shows you a very similar videos to the ones you watched, but does not show different topic videos. Watching cooking videos will cause the algorithms to recommend more cooking videos. However, if the algorithm recommends a different topic there is a risk that you will not find it interesting.


### Bayesian Ranking

Random listing of the items w.r.t underlying distribution. Let us say we list items by click through rate. We can treat CTR as a random number. How to score two distributions : 
* sample random numbers - Thompson Sampling 




### PageRank Algorithm

Prob that you will end up on a page if you surf on the internet randomly for the infinite amount of time.

Markov Models -- what is the probability that new page is x given that the current page is y. The next page depends on only the current page..

Consider page as a state at time t - $Pr(X_{t+1} | X_t)$

Transition matrix - probability of going one state to another $A_{i,j} = Pr(x_t = j| x_{t-1} = i)$. $$ \sum_{j=1}^M A_{ij} = \sum_{j=1}^M Pr(x_t = j| x_{t-1} =i) =1 $$.

Key point is that sum of transition probabilities in the same row must be 1. 

Example : Weather sunny and rainy day - from sunny to sunny or rainy, from rainty to sunny or rainy.
$Pr(sunny|rainy) = \frac{count(rainty->sunny)}{count(rainty)}$

Add one smoothing to avoid getting 0 probability if not observed rainy->sunny

* State distribution: $\pi^t =$ state probability distribution at time $t$.

Probability of being in a state at time $t$. Consider it as a row vector of state probabilities.$$\pi^{t+n} = \pi^{t}*A^n\\
pi^\infty = \lim_{t\to\infty}\pi_0A^t = \pi^\infty A\\
\pi = \pi A$$ where $\pi$ is the limiting probability distribution. 
This is just the eigen value problem. Matrix and a vector and a number : multiplying matrix by a vector is equal to vector by a number. -- multiplying vector by a matrix will not change its direction but magnitude $Av = \lambda v $.

The problem $\pi = \pi A$ is the same as eigen value problem where eigen value is one. The difference is that state distribution is a row vector hence it is on the left rather than on the right as in the eigen vector formula.

So think of the pages as the states, and we can calculate transition probs between the pages that all the links apper on the page (equal prob: $1/n_{i}$). Since there are million of pages, the matrix will be a sparse matrix. So we will smooth the the probs. $G = 0.85*A + 0.15U$ where $A$ is the transition matrix and $U$ is the similar matrix where $U_{i,j} = 1/M$

## Evaluating Recommendations

Interested in returning the list with highest ranking at the top and lowest at the bottom: [wiki](https://en.wikipedia.org/wiki/Learning_to_rank#Evaluation_measures)

More realistic: Revenues - which is related to clicks and impressions. (Deploy and evaluate) A/B test.

## Collaborative filtering

$s_{i,j}$ is the rating of item $j$ for user $i$. If we use thea average rating weighted by the user-user similarity vectors, we would use $$ s_{i,j} = \frac{\sum_{i'\inΩ_j}w_{i,i'}r_{i',j}}{\sum_{i'\in\Omega_j} w_{i,i'}} $$.

where $r_{i,j}$ is an element of $R_{NxM}$ which is a sparse user-item matrix. Sparsity means most of the elements is missing or empty. The goal is to predict the missing elements/ratings and recommending the highest rating items to the user.

$$MSE = \frac{1}{\Omega} \sum_{i,j\in Ω}(r_{i,j}-\hat{r}_{i,j})^2$$

Users may be pessimistic or optimistic - tend to give smaller or higher ratings. (how much user rating deviates from his/her rating).
$$dev_{i,j} = r_{i,j} - \bar{r}\text{  For known ratings}\\
\hat{dev_{i,j}} = \frac{1}{\Omega_{i'}}\frac{\sum_{i'\in \Omega_j}w_{i,i'}(r_{i',j}-\bar{r}_i')}{\sum_{i'\in \Omega_j}|w_{i,i'}|}\text{  For a prediction from known ratings}\\
\hat{s_{i,j}} = \bar{r_i}+\hat{dev_{i,j}}$$

Simple way to predict similarity weights $w_{i,i'}$ is to use cosine similarity or pearson similarities. 

To make the calculations faster, we can only consider the k-nearest neighbors, or we can ignore the users with only small number of ratings.

## Data Preprocessing

The [data](https://www.kaggle.com/datasets/grouplens/movielens-20m-dataset?select=rating.csv) has 20M rows. Each row contains user id, movie id, and a rating between 0 and 5. The goal is to get the data into user item matrix format.

In [ ]:
import zipfile
zipref = zipfile.ZipFile('movielens_ratings.zip')
zipref.extractall()
zipref.close()

In [ ]:
import pandas as pd
ratings = pd.read_csv('rating.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [ ]:
# Get user and movie counts
N = ratings.userId.nunique()
M = ratings.movieId.nunique()

print(f'There are {ratings.shape[0]} ratings in the data')
print(f'There are {N} users')
print(f'Max and Min user Ids are {ratings.userId.max()}, {ratings.userId.min()}')
print(f'There are {M} movies')
print(f'Max and Min movie Ids are {ratings.movieId.max()}, {ratings.movieId.min()}')

There are 20000263 ratings in the data
There are 138493 users
Max and Min user Ids are 138493, 1
There are 26744 movies
Max and Min movie Ids are 131262, 1


In [ ]:
from collections import Counter

user_ids_count = Counter(ratings.userId)
movie_ids_count = Counter(ratings.movieId)

# number of users and movies we would like to keep
n = 10000
m = 2000

user_ids = [u for u, c in user_ids_count.most_common(n)]
movie_ids = [m for m, c in movie_ids_count.most_common(m)]

In [ ]:
# Get the subset of ratings data with n users and m movies
subset = ratings[(ratings.movieId.isin(movie_ids))&(ratings.userId.isin(user_ids))]
subset.shape

(5392025, 4)

In [ ]:
# Create user and movies dictionaries
user_index = {i:j for j,i in enumerate(user_ids)}
movie_index = {i:j for j,i in enumerate(movie_ids)}

In [ ]:
# Create train and test sets 
from sklearn.model_selection import train_test_split
train, test = train_test_split(subset, test_size = 0.2, shuffle = True)
train.shape, test.shape

((4313620, 4), (1078405, 4))

# Build the ranking by user-user similarities

We want to calculate the following formula for the users. $$\hat{dev_{i,j}} = \frac{\sum_{i'\in \Omega_j}w_{i,i'}(r_{i',j}-\bar{r}_i')}{\sum_{i'\in \Omega_j}|w_{i,i'}|}\text{  For a prediction from known ratings}\\
\hat{s_{i,j}} = \bar{r_i}+\hat{dev_{i,j}}$$
* Steps:
1. Fill in the user-item matrix using train set (it would be better to use sparse matrix, but since I am working with a smaller subset, it is not necessary)
2. Calculate user similarity matrix
3. Calculate user average rating and scale the ratings -> rating deviance
4. Calculate the score formula

In [ ]:
# Create User-Item matrix
from tqdm import tqdm
import numpy as np
user_item_matrix = np.zeros(shape = (n,m))
for row in tqdm(train.values):
  user_item_matrix[user_index[row[0]], movie_index[row[1]]] = row[2]
user_item_matrix.shape

100%|██████████| 4313620/4313620 [00:07<00:00, 562213.53it/s]


(10000, 2000)

In [ ]:
# Calculate the average user rating and shift ratings
user_average  = np.sum(user_item_matrix, axis = 1, keepdims = True)/np.sum(user_item_matrix!=0, axis = 1, keepdims = True)
print(user_average.shape, user_average.min(), user_average.max())
user_item_matrix_scaled = np.where(user_item_matrix!=0, user_item_matrix - user_average,0)
print(user_item_matrix_scaled.shape, user_item_matrix_scaled.min(), user_item_matrix_scaled.max())

(10000, 1) 1.132034632034632 4.927007299270073
(10000, 2000) -4.074534161490683 3.6674107142857144


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
# Calculate cosine similarity between users
user_similarity = cosine_similarity(user_item_matrix_scaled)

In [ ]:
# Create a similarity mask (will ignore similarity if the number of movies rated by both users is less than a cutoff value)
mask = (user_item_matrix_scaled!=0).astype('float32')
masked_similarity = mask@mask.T

In [ ]:
# Calculate weighted score matrix using top similar users
user_similarity_norm = user_similarity.copy()
user_similarity_norm = np.where(masked_similarity<5, 0, user_similarity_norm) # keep most confident similarities

# Select top k similar users
k = 25
top_indices = np.argpartition(user_similarity_norm, -k, axis=1)[:, -k:]

# Set the other values to zero in place
for i in range(n):
    not_top_indices = np.delete(np.arange(n), top_indices[i])
    user_similarity_norm[i, not_top_indices] = 0

# diagonal values are not useful for making recommendations
for i in range(n):
  user_similarity[i,i]=0 

# Scale the weights to sum up to 1
user_similarity_sum = user_similarity_norm.sum(axis=1) # get row sum
user_similarity_sum[user_similarity_sum == 0] = 1 # if zero set to 1 to avoid division by zero
user_similarity_norm /= user_similarity_sum[:, np.newaxis]  # divide by sum to get similarity weights
user_similarity_norm.shape, user_similarity_norm.max(), user_similarity_norm.min()

((10000, 10000), 0.3846965212388212, 0.0)

In [ ]:
del top_indices
del user_similarity_sum
del subset
del ratings
del user_ids_count
del movie_ids_count
del user_ids
del movie_ids

In [ ]:
# Calculate the score matrix
score_matrix = np.matmul(user_similarity_norm, user_item_matrix_scaled) + user_average  # get rating predictions using similar user ratings
score_matrix = np.where(score_matrix<0.5, 0.5, score_matrix)  # ratings range between 0.5 and 5 - if interested in ranking then no need to truncate
score_matrix = np.where(score_matrix>5, 5, score_matrix) 
score_matrix.shape, score_matrix.max().max(), score_matrix.min().min() # confirm the score range

((10000, 2000), 5.0, 0.5)

In [ ]:
del user_similarity
del user_similarity_norm

In [ ]:
# Predict test data 
user_preds = []
for row in test.values:
  user_preds.append(score_matrix[user_index[row[0]], movie_index[row[1]]])

In [ ]:
# Mae for test data
from sklearn.metrics import mean_absolute_error, mean_squared_error
mean_absolute_error(test.rating.values, user_preds) , mean_squared_error(test.rating.values, user_preds)

(0.6322528683102926, 0.6813177292335322)

# Item-Item Collaborative filtering

In [ ]:
# Create User-Item matrix
from tqdm import tqdm
import numpy as np
user_item_matrix = np.zeros(shape = (m,n))
for row in tqdm(train.values):
  user_item_matrix[movie_index[row[1]], user_index[row[0]]] = row[2]
print(user_item_matrix.shape)

# Calculate the average movie rating
movie_average = np.sum(user_item_matrix, axis = 1, keepdims = True)/np.sum(user_item_matrix!=0, axis = 1, keepdims = True)
print(movie_average.shape, movie_average.min(), movie_average.max())

# Scale the matrix
user_item_matrix_scaled = np.where(user_item_matrix!=0, user_item_matrix - movie_average,0)
print(user_item_matrix_scaled.shape, user_item_matrix_scaled.min(), user_item_matrix_scaled.max())

100%|██████████| 4313620/4313620 [00:08<00:00, 505120.63it/s]


(2000, 10000)
(2000, 1) 1.4895031490552835 4.39356022689356
(2000, 10000) -3.89356022689356 3.5104968509447163


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
# Calculate cosine similarity between users
movie_similarity = cosine_similarity(user_item_matrix_scaled)

In [ ]:
# Create a similarity mask (will ignore similarity if the number of movies rated by both users is less than a cutoff value)
mask = (user_item_matrix_scaled!=0).astype('float32')
masked_similarity = mask@mask.T<5

# Calculate weighted score matrix using top similar users
movie_similarity_norm = movie_similarity.copy()
movie_similarity_norm = np.where(masked_similarity, 0, movie_similarity_norm) # keep most confident similarities

# Select top 100 similar movies
k = 25
top_indices = np.argpartition(movie_similarity_norm, -k, axis=1)[:, -k:]

# Set the other values to zero in place
for i in range(m):
    not_top_indices = np.delete(np.arange(m), top_indices[i])
    movie_similarity_norm[i, not_top_indices] = 0

# diagonal values are not useful for making recommendations
for i in range(m):
  movie_similarity_norm[i,i]=0 

# Scale the weights to sum up to 1
movie_similarity_sum = movie_similarity_norm.sum(axis=1)
movie_similarity_sum[movie_similarity_sum == 0] = 1
movie_similarity_norm /= movie_similarity_sum[:, np.newaxis]
movie_similarity_norm.shape, movie_similarity_norm.max(), movie_similarity_norm.min()

((2000, 2000), 0.16309781562664033, 0.0)

In [ ]:
# Calculate the score matrix
score_matrix = np.matmul(movie_similarity_norm, user_item_matrix_scaled) + movie_average
score_matrix = np.where(score_matrix<0.5, 0.5, score_matrix)
score_matrix = np.where(score_matrix>5, 5, score_matrix)
score_matrix.shape, score_matrix.max().max(), score_matrix.min().min()

((2000, 10000), 5.0, 0.5)

In [ ]:
# Predict test data 
preds = []
for row in test.values:
  preds.append(score_matrix[movie_index[row[1]],user_index[row[0]]])
  
# Mae for test data
from sklearn.metrics import mean_absolute_error, mean_squared_error
mean_absolute_error(test.rating.values, preds) , mean_squared_error(test.rating.values, preds)

(0.6019839345947412, 0.6163038410564982)

Item-item recommender performs better than user-user recommender because we have relatively less movies than users, so we have more values to find accurate similarities between movies.

# Find the weighted rating predictions using user2user and movie2movie predictions


In [ ]:
# Find the weight that results in the lowest mean absolute error
threshold = np.linspace(0.2, 0.9, 30)
errors = []
lowest_error = 100
best_threshold = 0
for i in threshold:
  mean_preds = ((1-i)*np.array(user_preds)+i*np.array(preds))
  error = mean_absolute_error(test.rating.values, mean_preds)
  if error<lowest_error:
    best_threshold = i
    lowest_error = error
print(best_threshold, lowest_error)

0.7310344827586206 0.5969067126717832
